# Chess positions and knight's tours via graphs

<span style="font-size: 16pt; font-style: italic; font-weight: bold">...in Raku</span>

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
[RakuForPrediction-book at GitHub](https://github.com/antononcube/RakuForPrediction-book)      
November-December 2024

-----

## Introduction

In this notebook (presentation) we discuss:

- Two ways of plotting chessboards
- [Knight's tour](https://en.wikipedia.org/wiki/Knight%27s_tour) graphs
- [Hamiltonian paths](https://en.wikipedia.org/wiki/Hamiltonian_path)
- Animations of knight's tours

Showcased:
- All computational graph features discussed here are provided by ["Graph"](https://raku.land/zef:antononcube/Graph).  
- The chessboard plots with:
    - `dot-chessboard`, provided by ["Graphviz::DOT::Chessboard"](https://raku.land/zef:antononcube/Graphviz::DOT::Chessboard)
    - `js-d3-chessboard`, provided by ["JavaScript::D3"](https://raku.land/zef:antononcube/JavaScript::D3).
- Graph plotting with:
    - `js-d3-graph-plot`, provided by ["JavaScript::D3"](https://raku.land/zef:antononcube/JavaScript::D3).
    -  `Graph.dot`, that makes SVG images via [Graphviz](https://graphviz.org).

------

## Setup

Here are loaded the packages used in the rest of notebook:

In [ ]:
use Graph;

use Graph::Circulant;
use Graph::Complete;
use Graph::CompleteKaryTree;
use Graph::Cycle;
use Graph::Grid;
use Graph::HexagonalGrid;
use Graph::Hypercube;
use Graph::KnightTour;
use Graph::Nested;
use Graph::Path;
use Graph::Petersen;
use Graph::Star;
use Graph::TriangularGrid;
use Graph::Wheel;

use Graph::Distribution;
use Graph::Random;

use Data::Reshapers;
use Data::Summarizers;
use Data::Generators;
use Data::TypeSystem;
use Data::Translators;
use Data::Geographics;

use Math::DistanceFunctions;
use Math::Nearest;
use Text::Levenshtein::Damerau;

use Hash::Merge;
use FunctionalParsers;
use FunctionalParsers::EBNF;
use EBNF::Grammar;
use Graphviz::DOT::Grammar;
use Graphviz::DOT::Chessboard;

use JavaScript::D3;
use WWW::MermaidInk;

use paths;

### JavaScript

Here we prepare the notebook to visualize with JavaScript:

In [ ]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

Verification:

In [ ]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

Here we set a collection of visualization variables:

In [ ]:
my $title-color = 'Ivory';
my $tick-labels-color = 'Ivory';
my $stroke-color = 'SlateGray';
my $tooltip-color = 'LightBlue';
my $tooltip-background-color = 'none';
my $background = '#1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $vertex-size = 6;
my $engine = 'neato';
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

my %opts = :$background, :$title-color, :$edge-thickness, :$vertex-size;

### LLM configurations

In [ ]:
my $conf4o = llm-configuration('chatgpt', model=>'gpt-4o', max-tokens=>4096, temperature=>0.45);
my $conf-gemini = llm-configuration('gemini', model=>'gemini-1.5-pro-latest', max-tokens=>4096, temperature=>0.45);

($conf4o, $conf-gemini).map(*.Hash.elems)

------

## Chessboards

### SVG plots via **Graphviz DOT**

Using the package ["Graphviz::DOT::Chessboard"](), [AAp2].

In [ ]:
#%html
my %opts-brown = black-square-color => '#b68864', white-square-color => '#f1d9b5';
my %opts-green = black-square-color => '#779556ff', white-square-color => '#ebecd0ff', white-piece-stroke-color => 'LightGray';
my %opts = |%opts-brown, :4r, :6c, size => (6,6);
my $dot = dot-chessboard(|%opts);
dot-chessboard(|%opts):svg

In [ ]:
#% html
dot-chessboard(<a1N b3q>».comb, :4r, :6c, |%opts):svg;

### JavaScript plots

Here is another way to make a chessboard using ["JavaScript::D3"](https://raku.land/zef:antononcube/JavaScript::D3), [AAp3]:

In [ ]:
#% js
js-d3-chessboard('8/8/8/8/8/8/8/8', :$background, :$tick-labels-color, :$title-color, width => 500, color-palette => 'Greys')

Here we specify position of the pieces:

In [ ]:
#% js
js-d3-chessboard(<a1N b3q>».comb».List, :$background, :$tick-labels-color, width => 500);

**Remark:** The JavaScript (D3.js) plots are not used here because it is hard to do combined board-tour plots like the one shown below.

### FEN strings

Both packages -- "Graphviz::DOT::Chessboard" and "JavaScript::D3" -- can process Forsyth-Edwards Notation (FEN) strings of chess positions:

In [ ]:
#%html
my $fen = '8/8/8/3K4/5r2/8/1k6/8';

dot-chessboard($fen, :8r, :8c, :4size):svg

In [ ]:
#% js
js-d3-chessboard($fen, :$background, :$tick-labels-color,  width => 500)

### Small-chess initial position

Here is the initial position of the a "quick", 5x5 chess:

In [ ]:
#%html
my $fen = '8/8/8/rnbqk3/ppppp3/8/PPPPP3/RNBQK3';

dot-chessboard($fen, :5r, :5c, :4size, |%opts-green):svg

-----

## Chessboard plotting steps

This section gives step-by-step explanations how to make chessboard plots using grid graphs and Graphviz.

Make a chessboard SVG plot using a grid graph:

In [ ]:
#%html
my $gg = Graph::Grid.new(4, 6);

# Leveraging that grid graphs are bipartite
my %highlight = <DimGray LightGray> Z=> $gg.bipartite-coloring.classify(*.value).nodemap(*».key).values;


# DOT language spec
my $preamble = q:to/END/;
fontcolor = "Ivory";
fontsize = "16";
labelloc = "t";
label = "Chessboard by DOT";
graph [size="6,12!"];

bgcolor="#1F1F1F";
node [style=filled, label="", opacity=0.3, fixedsize=true, shape=square, color="Black", fillcolor="SlateBlue", penwidth=1, fontsize=4, fontcolor="White", labelloc=c, width=0.98, height=0.3];
edge [style=invis, color="SteelBlue", penwidth=0.6];
END

# To be reused for overlaying tour paths
my $dot = $gg.dot(:$preamble, :%highlight, :!node-labels).lines[1..*-2].join("\n");

# Rendering of the DOT spec
$gg.dot(:$preamble, :%highlight, :!node-labels, engine=>'neato'):svg;

--------

## Knight's tour over a chessboard

Make a Knight's tour graph:

In [ ]:
my $gkt = Graph::KnightTour.new(4, 6, prefix => 'kt');

Show the corresponding Graphviz DOT plot:

In [ ]:
#% html
$gkt.dot(engine => 'neato', :$background, vertex-shape => 'ellipse', vertex-width => 0.6):svg

-----

## Hamiltonian path finding

- Finding a Hamiltonian path is NP-complete problem. 
- Hence, using algorithms that is fast, but do not always "work" can be desirable.

Two algorithms to consider:

- The backtracking algorithm with [Warnsdorf's rule](https://en.wikipedia.org/wiki/Knight%27s_tour#Warnsdorf's_rule)
- Random (Angluin-Valiant) algorithm

Using (deterministic) backtracking algorithm to find a Hamiltonian path:

In [ ]:
my $end = 'kt0_3'; #'kt0_3';# Whatever;
my @hpath = $gkt.find-hamiltonian-path('kt0_0', $end, method => 'backtracking', :warnsdorf-rule);

Using (random) Angluin-Valiant algorithm:

In [ ]:
my @hpath = $gkt.find-hamiltonian-path('kt0_0', $end, method => 'random', pick => 'max-degree');

Show the found Hamiltonian Knight path on a chessboard:

In [ ]:
#% html

# Reusing the "chessboard graph" DOT spec above.
my $preamble = [
    $dot.subst('--', '->', :g).subst('label="",').subst('Chessboard by DOT', "Knight's Hamiltonian path"),
    'node [shape=ellipse, fillcolor=Yellow, width=0.2, height=0.2, fixedsize=true, style=filled, color=Orange, fontsize=10, fontcolor=Black];',
    'edge [style=filled, width=103, color=Orange];',
].join("\n");

# Make a path graph from the found Hamiltonian path.
my $gp = Graph::Path.new(@hpath):directed;

# Set the graph vertex coordinates.
$gp.vertex-coordinates = $gkt.vertex-coordinates;

# Replace (rename) the path graph vertices to ordinals. 
# (In order to show the sequential order of the path.)
$gp .= vertex-replace((@hpath Z=> (^@hpath.elems)».Str).Hash); 

my $kdot = $gp.dot(:$preamble, engine=>'neato'):svg;

Why not use the following?

In [ ]:
@hpath.kv.Hash.invert.raku

-----

## Animation

It would be nice to produces an animation of the knight's tour. 

***It is an instructive exercise to figure out how to make such an animation using SVG, LLMs (in Raku.)***

### Get path points

Extract the knight jump points from SVG plot:

In [ ]:
my @points = do with $kdot.match(/ '<title>' (\d+) '</title>' \s* '<ellipse fill="Yellow" stroke="Orange" cx="' (.*?) '" cy="' (.*?) '"'/, :g) {
    $/.map({ $_.values».Str })
}

Sort in the order of jumping:

In [ ]:
@points = @points».Numeric.sort(*.head).map(*.tail(2))».Array.Array

### LLM function

Make an LLM function that produces SVG animation specs for given lists of points:

In [ ]:
my &fAnim = llm-function({
    [ 
        llm-prompt('CodeWriterX')('SVG'),
        "Make an animation of a red circle along these points (x, y):",
        '```raku',
        $_.raku,
        '```',
        'Use the SVG `<path>` element.',
        'Make it adhere to an element that has the following transformation:',
        '```svg',
        '<g id="anim" class="graph" transform="scale(1.31339 1.31339) rotate(0) translate(4 317.31)">',
        '```',
        'Make it stop at the last point.'
    ]},
    e => $conf4o
)

### Make the animation fragment

In [ ]:
# my $res = &fAnim(@points).match(/'<g' .* '</g>'/).Str

Combine the SVGs into an animation:

In [ ]:
#%html
# my $dot2 = $kdot.subst('</svg>', $res ~ "</svg>")

Instead of calling the LLM-function repeatedly let us make a dedicated one:

In [ ]:
my $transform = ($kdot ~~ / 'transform="scale(' .*? ')'/).Str

In [ ]:
sub animation-fragment(@points) {
    my $path = @points.map({"L" ~ $_.join(',')}).join("\n");
    $path = $path.subst('L', 'M');

    my $svg-anim = Q:s:to/END/;
    <g id="anim" class="graph" $transform rotate(0) translate(4 317.31)">
        <path id="path" fill="none" stroke="none" d="
        $path
        " />
        <circle r="6" fill="blue">
        <animateMotion dur="10s" repeatCount="1" fill="freeze">
            <mpath href="#path" />
        </animateMotion>
        </circle>
    </g>
    END
}

In [ ]:
#%html
my $dot2 = $kdot.subst('</svg>', animation-fragment(@points) ~ "</svg>")

-------

## Alternative animation

Here we show the first few Knight moves using chess-coordinates (or a FEN string):

In [ ]:
#%html
my @pos = @hpath.head(5)
    .map({ $_.subst('kt').split('_')».Int })
    .map({ ( ('a'..'h')[$_.head], ('1'..'8')[$_.tail], 'n') });
dot-chessboard(@pos, |%opts):svg

It is natural to think about think about animating the knight jump. 

Assuming that we know how to make SVG animations we can do the following:

- Make a chessboard background SVG image
- For each vertex of the knight's tour path make a subgraph and plot it as an SVG image
- Combine the background image and "jump" images into an SVG animation

The approach above, though, requires proper scaling of corresponding graphics. 

It is much easier to:

- For each knight jump make an SVG image of it with the whole chessboard
- Extract the graphics element from each image (`<g>`)
- Put the images together into an SVG animation specification

Each of the knight-jump-chessboards has the part knight tour from the start its jump.
Fore that we are re-doing the above with using `dot-chessboard` and the "orange board" styling: 

In [ ]:
#%html

my $preamble2 = [
    dot-chessboard(|%opts).lines[1..*-2].subst('--', '->', :g).subst('label="",').subst('Chessboard by DOT', ''),
    'node [shape=ellipse, fillcolor=Gray, width=0.2, height=0.2, fixedsize=true, style=filled, color=SlateBlue, fontsize=10, fontcolor=Black];',
    'edge [style=filled, width=103, color=SlateBlue];',
].join("\n");

my $gp2 = Graph::Path.new(@hpath):directed;
$gp2.vertex-coordinates = $gkt.vertex-coordinates;
$gp2 .= vertex-replace((@hpath Z=> (^@hpath.elems)».Str).Hash); 

# To verify
# $gp2.dot(preamble => $preamble2, engine=>'neato'):svg

"Partial tour" chessboards:

In [ ]:
my @knight-jumps = 
    (^$gp2.vertex-count)
    .map({ $gp2.subgraph((^$_)».Str).dot(preamble => $preamble2, engine=>'neato', |%opts):svg });

@knight-jumps.elems

Example:

In [ ]:
#% html
@knight-jumps[4]

Here we extract the graphics elements:

In [ ]:
my @knight-jumps2 = 
    @knight-jumps
    .map({
        do with $_.match(/ ('<g id="graph0" class="graph"' .* '</g>') \s* '</svg>' /) { $0.Str } 
    })
    .pairs.map({
        $_.value.subst(/ '"graph' \d+ '"' /, "\"graph{$_.key + 1}\"")
    });

@knight-jumps2.elems

Putting the images together into an SVG animation:

In [ ]:
my $view = (@knight-jumps.head ~~ / 'viewBox="' .*? '"' /).Str;
my $defs = @knight-jumps2.join("\n");
my $values = ('#graph' X~ (1..@knight-jumps.elems) X~ ';').join;

Q:s:to/END/;
<svg width="433pt" height="306pt" $view xmlns="http://www.w3.org/2000/svg">
  <defs>
    $defs
  </defs>

  <use href="#graph0" id="animatedImage">
    <animate attributeName="href" values="$values" dur="5s" repeatCount="1" fill="freeze"/>
  </use>
</svg>
END

**Remark:** The animation workflow is fairly universal and it is a good idea to make it into a standalone function that takes a list of SVG images and makes an animation with them.

------

## References

### Articles

[Wk1] Wikipedia entry, ["Knight's tour"](https://en.wikipedia.org/wiki/Knight%27s_tour).

[Wk2] Wikipedia entry, ["Hamiltonian path problem"](https://en.wikipedia.org/wiki/Hamiltonian_path_problem).

### Packages, paclets

[AAp1] Anton Antonvov,
[Graph Raku package](https://github.com/antononcube/Raku-Graph),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonvov,
[Graph Raku package](https://github.com/antononcube/Raku-Graphviz-DOT-Chessboard),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonvov,
[JavaScript::D3 Raku package](https://github.com/antononcube/Raku-JavaScript-D3),
(2022-2024),
[GitHub/antononcube](https://github.com/antononcube).

### Videos

[AAv1] Anton Antonov,
[Graph neat examples in Raku (Set 1)](https://youtu.be/5qXgqqRZHow),
(2024),
[YouTube/@AAA4prediction](https://www.youtube.com/@AAA4prediction).

[AAv2] Anton Antonov,
[Graph neat examples in Raku (Set 2)](https://youtu.be/E7qhutQcWCY),
(2024),
[YouTube/@AAA4prediction](https://www.youtube.com/@AAA4prediction).

[AAv3] Anton Antonov,
[Graph neat examples in Raku (Set 3)](),
(2024),
[YouTube/@AAA4prediction](https://www.youtube.com/@AAA4prediction).
